# Text Analytics with Python - Part 2

Text analytics, also known as Natural Language Processing (NLP), refer to the analytics toward text-based data.

Text-based communication has become one of the most common forms of expression. We email, text message, tweet, and update our statuses on a daily basis. As a result, unstructured text data has become extremely common, and analyzing large quantities of text data is now a key way to understand what people are thinking.

One of the biggest breakthroughs required for achieving any level of artificial intelligence is to have machines which can process text data. Thankfully, the amount of text data being generated in this universe has exploded exponentially in the last few years.

It has become imperative for an organization to have a structure in place to mine actionable insights from the text being generated. From social media analytics to risk management and cybercrime protection, dealing with text data has never been more important.

In this article we will discuss different feature extraction methods, starting with some basic techniques which will lead into advanced Natural Language Processing techniques. We will also learn about pre-processing of the text data in order to extract better features from clean data.

By the end of this article, you will be able to perform text operations by yourself. Let’s get started!

Original tutorial from [link](https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/), revised and adapted by Dr. Tao.
ver 1.0, May 2018

# Table of Contents

**[TODO] To be updated**

## Load data

In [ ]:
from nltk.corpus import twitter_samples
pos_tweets = twitter_samples.strings('positive_tweets.json')

In [ ]:
import pandas as pd
pos_tweets = pd.DataFrame(pos_tweets, columns=['text'])
pos_tweets.head()

# 3. Advance Text Processing

Up to this point, we have done all the basic pre-processing steps in order to clean our data. Now, we can finally move on to extracting features using NLP techniques.

## 3.1 N-grams

N-grams are the combination of multiple words used together. Ngrams with N=1 are called unigrams. Similarly, bigrams (N=2), trigrams (N=3) and so on can also be used.

Unigrams do not usually contain as much information as compared to bigrams and trigrams. The basic principle behind n-grams is that they capture the language structure, like what letter or word is likely to follow the given one. The longer the n-gram (the higher the n), the more context you have to work with. Optimum length really depends on the application – if your n-grams are too short, you may fail to capture important differences. On the other hand, if they are too long, you may fail to capture the “general knowledge” and only stick to particular cases.

So, let’s quickly extract bigrams from our tweets using the ngrams function of the textblob library.

In [ ]:
from textblob import TextBlob
TextBlob(pos_tweets['text'][0]).ngrams(2)

The value of obtaining n-grams is to capture the context (surrounding words) of any target word of analysis.

##3.2 Term frequency

Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

Therefore, we can generalize term frequency as:

**TF = (Number of times term T appears in the particular row) / (number of terms in that row)**

To understand more about Term Frequency, have a look at this article.

Below, I have tried to show you the term frequency table of a tweet.

In [ ]:
tf1 = (pos_tweets['text'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

##3.3 Inverse Document Frequency

The intuition behind inverse document frequency (IDF) is that a word is not of much use to us if it’s appearing in all the documents.

Therefore, the IDF of each word is the log of the ratio of the total number of rows to the number of rows in which that word is present.

**IDF = log(N/n)**, where, N is the total number of rows and n is the number of rows in which the word was present.

So, let’s calculate IDF for the same tweets for which we calculated the term frequency.



In [ ]:
import numpy as np
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(pos_tweets.shape[0]/(len(pos_tweets[pos_tweets['text'].str.contains(word, regex=False)])))

tf1

The more the value of IDF, the more **unique** is the word.

##3.4 Term Frequency – Inverse Document Frequency (TF-IDF)

TF-IDF is the multiplication of the TF and IDF which we calculated above. **TF-IDF** is a very important approach of quantifying textual data.

In [ ]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

We can see that the TF-IDF has penalized words like ‘and’, ‘on’, and ‘you’ because they are commonly occurring words. However, it has given a high weight to “disappointed” since that will be very useful in determining the sentiment of the tweet.

We don’t have to calculate TF and IDF every time beforehand and then multiply it to obtain TF-IDF. Instead, sklearn has a separate function to directly obtain it:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(pos_tweets['text'])

We can also perform basic pre-processing steps like lower-casing and removal of stopwords, if we haven’t done them earlier.

In [ ]:
train_vect[0]

## 3.5 Bag of Words

Bag of Words (BoW) refers to the representation of text which describes the presence of words within the text data. The intuition behind this is that two similar text fields will contain similar kind of words, and will therefore have a similar bag of words. Further, that from the text alone we can learn something about the meaning of the document.

For implementation, sklearn provides a separate function for it as shown below:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(pos_tweets['text'])
train_bow[0]

## 3.6 Sentiment Analysis

If you recall, our problem was to detect the sentiment of the tweet. So, before applying any ML/DL models (which can have a separate feature detecting the sentiment using the textblob library), let’s check the sentiment of the first few tweets.

In [ ]:
pos_tweets['text'][:10].apply(lambda x: TextBlob(x).sentiment)

Above, you can see that it returns a tuple representing polarity and subjectivity of each tweet. Here, we only extract polarity as it indicates the sentiment as value nearer to 1 means a positive sentiment and values nearer to -1 means a negative sentiment. This can also work as a feature for building a machine learning model.

In [ ]:
pos_tweets['sentiment'] = pos_tweets['text'].apply(lambda x: TextBlob(x).sentiment[0] )
pos_tweets[['text','sentiment']].head()

## 3.7 Word Embeddings

Word Embedding is the representation of text in the form of vectors. The underlying idea here is that similar words will have a minimum distance between their vectors.

Word2Vec models require a lot of text, so either we can train it on our training data or we can use the pre-trained word vectors developed by Google, Wiki, etc.

Here, we will use pre-trained word vectors which can be downloaded from the glove website. There are different dimensions (50,100, 200, 300) vectors trained on wiki data. For this example, I have downloaded the 100-dimensional version of the model.

The first step here is to convert it into the word2vec format.

In [ ]:
# code will take too long to run, not running it.